# Работа модели для матчинга резюме и вакансий портала "Работа в России":

In [ ]:
#Установим библиотеку, разработанную ЦПУР
#jupyter notebook --NotebookApp.allow_origin='https://colab.research.google.com' --port=8899 --NotebookApp.port_retries=0
!git clone https://github.com/ellariel/rostrud_ml.git

Cloning into 'rostrud_ml'...


После скачивания корневая папка должна быть переименована в rostrud_ml

## Предварительные действия
У этого скрипта есть несколько зависимостей, их можно поставить через pip. При запуске блока кода ниже всё поставится само.

Если в какой-то момент выполнения кода вы увидите ошибку, что вам не хватает некого питоновского пакета, вы можете дописать его название в новую строчку файла requirements.txt и снова запустить этот блок.

In [ ]:
#Установка окружения
!pip install -r rostrud_ml/requirements.txt

В файле rostrud_ml/utils/all_tables_names.yml указать путь к директории, куда будут скачиваться и парситься файлы с резюме, вакансиями (сейчас там "workdir").

В файле rostrud-ml/utils/config_to_bd_example.yml указать ваши настройки для подключения к БД и поменять название файла на config_to_bd.yml

In [1]:
import requests
from tqdm import tqdm
import pandas as pd

In [2]:
base_url = 'https://opendata.trudvsem.ru/oda2Hialoephidohyie1oR6chaem1oN0quiephooleiWei1aiD/'
file_url = ['7710538364-cv/', '7710538364-professions/', '7710538364-regions/', '7710538364-industries/', '7710538364-vacancy/', '7710538364-invitation/']

In [ ]:
# last file in each month
def _getfileslist(url):
  r = requests.get(url)
  return ['data' + i.split('">data')[0] for i in r.text.split('<a href="data')[1:]]

s = pd.DataFrame(_getfileslist(base_url + file_url[0])).rename(columns={0: 'cv'})
s = s[s['cv'].apply(lambda x: pd.notna(x) and '.gz' in x)]
s = s[s['cv'].apply(lambda x: int(x.split('data-')[1][:4]) >= 2019)]
s['month'] = s['cv'].apply(lambda x: x.split('data-')[1][:6])
s = s.groupby('month').agg('last').reset_index()
for l in file_url[1:]:
    c = l.split('-')[1]
    c = c[:len(c)-1]
    v = pd.DataFrame(pd.Series(_getfileslist(base_url + l)).rename(c))
    v = v[v[c].apply(lambda x: pd.notna(x) and '.gz' in x)]
    v = v[v[c].apply(lambda x: int(x.split('data-')[1][:4]) >= 2019)]
    v['month'] = v[c].apply(lambda x: x.split('data-')[1][:6])
    v = v.groupby('month').agg('last').reset_index()
    s = s.set_index('month').join(v.set_index('month')).reset_index()
s.to_csv('filtered_month.csv', index=False)

# all files
s = pd.DataFrame(_getfileslist(base_url + file_url[0])).rename(columns={0: 'cv'})
s = s[s['cv'].apply(lambda x: pd.notna(x) and '.gz' in x)]
s = s[s['cv'].apply(lambda x: int(x.split('data-')[1][:4]) >= 2019)]
s['date'] = s['cv'].apply(lambda x: x.split('data-')[1][:8])
s = s.groupby('date').agg('last').reset_index()
for l in file_url[1:]:
    c = l.split('-')[1]
    c = c[:len(c)-1]
    v = pd.DataFrame(pd.Series(_getfileslist(base_url + l)).rename(c))
    v = v[v[c].apply(lambda x: pd.notna(x) and '.gz' in x)]
    v = v[v[c].apply(lambda x: int(x.split('data-')[1][:4]) >= 2019)]
    v['date'] = v[c].apply(lambda x: x.split('data-')[1][:8])
    v = v.groupby('date').agg('last').reset_index()
    s = s.set_index('date').join(v.set_index('date')).reset_index()
s.to_csv('filtered_all.csv', index=False)

## Загрузка в БД необходимых данных

In [4]:
filtered = pd.read_csv('filtered_all.csv') #список ссылок полученный на прошлом этапе
len(filtered)

56

In [5]:
#Установка модулей
from rostrud_ml.process.adding_tables_psycopg import AddingDataPsycopg
from rostrud_ml.process.renewal import Renewal

Вы подключены к базе: postgres


In [6]:
#Cоздание таблиц в БД
db = AddingDataPsycopg()
for table in ['curricula_vitae', 'workexp', 'vacancies', 'professions', 'industries', 'regions', 'invitations']:
    db.create_table(table, 'project_trudvsem') #'project_trudvsem' - название схемы в вашей БД, создать, если нету
db.conn.close()

Вы подключены к базе: postgres


Взять ссылки на файлы с данными с ftp поратала https://opendata.trudvsem.ru/oda2Hialoephidohyie1oR6chaem1oN0quiephooleiWei1aiD/:

Из директорий
https://opendata.trudvsem.ru/oda2Hialoephidohyie1oR6chaem1oN0quiephooleiWei1aiD/7710538364-cv/
https://opendata.trudvsem.ru/oda2Hialoephidohyie1oR6chaem1oN0quiephooleiWei1aiD/7710538364-vacancy/ нужно выбрать подходящие файлы (имеющие в наименовании дату) - один файл резюме и один вакансий.

Подобным образом выбрать ссылки для данных о профессиях, сферах занятости, регионах из соответствующих директорий:
https://opendata.trudvsem.ru/oda2Hialoephidohyie1oR6chaem1oN0quiephooleiWei1aiD/7710538364-professions/
https://opendata.trudvsem.ru/oda2Hialoephidohyie1oR6chaem1oN0quiephooleiWei1aiD/7710538364-industries/
https://opendata.trudvsem.ru/oda2Hialoephidohyie1oR6chaem1oN0quiephooleiWei1aiD/7710538364-regions/

Для каждого типа, вставив нужную ссылку, запустить следующие ячейки:
В примере: тип "резюме" и самый первый файл резюме 2018г. Необходимо поменять ссылку на наиболее актуальный файл.

In [7]:
filtered.tail(1)

,month,cv,professions,regions,industries,vacancy,invitation
55,202308,data-20230830T053225-structure-20161130T143000...,data-20230831T081254-structure-20161130T143000...,data-20230831T081254-structure-20161130T143000...,data-20230831T081254-structure-20161130T143000...,data-20230830T214915-structure-20161130T143000...,data-20230831T093831-structure-20161130T143000...


In [ ]:
for _, link in tqdm(filtered.iterrows(), total=len(filtered)):
  if int(link['month']) < 202107:
    continue
  try:
    print('cv')
    ren = Renewal('cvs', base_url + file_url[0] + link['cv'])
    ren.download()
    ren.extract()
    ren.parse_update()
  except Exception as e:
    print(f'cvs error: {e}')
  ren.delete(remove_gz=True)

  if pd.notna(link['vacancy']):
    try:
      print('vacancies')
      ren = Renewal('vacancies', base_url + file_url[4] + link['vacancy'])
      ren.download()
      ren.extract()
      ren.parse_update()
    except Exception as e:
      print(f'vacancies error: {e}')
    ren.delete(remove_gz=True)

  if pd.notna(link['invitation']):
    try:
      print('invitations')
      ren = Renewal('invitations', base_url + file_url[5] + link['invitation'])
      ren.download()
      ren.extract()
      ren.parse_update()
    except Exception as e:
      print(f'invitations error: {e}')
    ren.delete(remove_gz=True)

  #try:
  #  print('professions')
  #  ren = Renewal('professions', base_url + file_url[1] + link['professions'])
  #  ren.download()
  #  ren.extract()
  #  ren.parse_update()
  #except Exception as e:
  #  print(f'professions error: {e}')
  #ren.delete(remove_gz=False)

  #try:
  #  print('industries')
  #  ren = Renewal('industries', base_url + file_url[2] + link['regions'])
  #  ren.download()
  #  ren.extract()
  #  ren.parse_update()
  #except Exception as e:
  #  print('industries error')
  #ren.delete(remove_gz=False)

  #try:
  #  print('regions')
  #  ren = Renewal('regions', base_url + file_url[3] + link['industries'])
  #  ren.download()
  #  ren.extract()
  #  ren.parse_update()
  #except Exception as e:
  #  print('regions error')
  #ren.delete(remove_gz=False)

  #break

  0%|                                                                                           | 0/56 [00:00<?, ?it/s]

cv
Файлы скачаны: 2021-07-31
Файлы извлечены: 2021-07-31
Вы подключены к базе: postgres
return hashes
df резюме готов
curricula_vitae : обработано
Вы подключены к базе: postgres
